In [66]:
!pip install torch

In [0]:
import pandas as pd
import numpy as np
import torch
import shutil
import os

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

In [0]:
batch_size = 32
learning_rate = 0.001
epochs = 20

In [0]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(14*14*256, 1024)
        self.fc2 = nn.Linear(1024, 2)
        self.relu  = nn.ReLU()
        self.out = nn.LogSoftmax(dim = 1)
        self.dropout = nn.Dropout(p = 0.3)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.out(x)
        return x

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [0]:
from torchvision import models
model_transfer = models.vgg16(pretrained=True)

In [0]:
for param in model_transfer.parameters():
    param.requires_grad = False

In [0]:
model = CNNModel().to(device)
loss = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
source = 'drive/My Drive/upload.zip'
destination = 'upload.zip'
shutil.copy(source, destination)

'upload.zip'

In [36]:
!unzip upload.zip

Archive:  upload.zip
replace upload/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
def load_dataset():
    transform_train = torchvision.transforms.Compose([
        #cropping and resizing the image to 224*224
        torchvision.transforms.Resize(224),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.RandomRotation(30),
        #convert image to tensor
        torchvision.transforms.ToTensor(),
        #normalizing the input mean = 0.5 and std = 0.5 for three channels (R,G,B)
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    dataset_train = torchvision.datasets.ImageFolder(
        root='upload/train/',
        transform=transform_train
    )
    dataset_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    transform_test = torchvision.transforms.Compose([
        torchvision.transforms.Resize(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    dataset_test = torchvision.datasets.ImageFolder(
        root='upload/test/',
        transform=transform_test
    )
    dataset_loader_test = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return dataset_loader_train, dataset_loader_test

In [0]:
def evaluate(dataset):
    with torch.no_grad():
        correct, total, loss_error, iterations = 0, 0, 0, 0
        for images, labels in dataset:
            outputs = model(images.cuda())
            errors = loss(outputs, labels.cuda())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.cuda()).sum().item()
            loss_error += errors.item()
            iterations += 1 
        accuracy = (correct / total) * 100
        loss_error = loss_error / iterations
        return accuracy, loss_error

In [81]:
loss_history = []
accuracy_history = []

dataset_train, dataset_test = load_dataset()
batches = len(dataset_train)

for j in range(0, epochs):
    for i, (images, labels) in enumerate(dataset_train):
        outputs = model(images.cuda())
        error = loss(outputs, labels.cuda())
        loss_history.append(error.item())
        
        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        error.backward()
        optimizer.step()
        
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.cuda()).sum().item()
        accuracy_history.append(correct / total)

        if (i + 1) % 219 == 0:
            test_accuracy, test_error = evaluate(dataset_test)
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, Test-loss: {:.4f}, Test-accuracy: {:.2f}%'
                  .format(j + 1, epochs, i + 1, batches, error.item(),
                          (correct / total) * 100, test_error, test_accuracy))

Epoch [1/20], Step [219/219], Loss: 0.0485, Accuracy: 95.92%, Test-loss: 0.3294, Test-accuracy: 90.68%
Epoch [2/20], Step [219/219], Loss: 0.0544, Accuracy: 100.00%, Test-loss: 0.1888, Test-accuracy: 92.74%
Epoch [3/20], Step [219/219], Loss: 0.0557, Accuracy: 97.96%, Test-loss: 0.1134, Test-accuracy: 95.91%
Epoch [4/20], Step [219/219], Loss: 0.0330, Accuracy: 97.96%, Test-loss: 0.1366, Test-accuracy: 96.26%
Epoch [5/20], Step [219/219], Loss: 0.0312, Accuracy: 100.00%, Test-loss: 0.1149, Test-accuracy: 95.97%
Epoch [6/20], Step [219/219], Loss: 0.0502, Accuracy: 97.96%, Test-loss: 0.0961, Test-accuracy: 95.97%
Epoch [7/20], Step [219/219], Loss: 0.0217, Accuracy: 100.00%, Test-loss: 0.1275, Test-accuracy: 94.94%
Epoch [8/20], Step [219/219], Loss: 0.0199, Accuracy: 97.96%, Test-loss: 0.1357, Test-accuracy: 95.83%
Epoch [9/20], Step [219/219], Loss: 0.0155, Accuracy: 100.00%, Test-loss: 0.0931, Test-accuracy: 96.40%
Epoch [10/20], Step [219/219], Loss: 0.0692, Accuracy: 95.92%, Test-l

In [82]:
evaluate(dataset_test)

(99.08545298656759, 0.023567325651476328)

In [40]:
source = 'drive/My Drive/prediction.zip'
destination = 'prediction.zip'
shutil.copy(source, destination)
!unzip prediction.zip

Archive:  prediction.zip
   creating: prediction/
  inflating: prediction/.DS_Store    
   creating: __MACOSX/prediction/
  inflating: __MACOSX/prediction/._.DS_Store  
   creating: prediction/prediction/
  inflating: prediction/prediction/7d8968a4b5ba035660101e6728a137a2.jpg  
   creating: __MACOSX/prediction/prediction/
  inflating: __MACOSX/prediction/prediction/._7d8968a4b5ba035660101e6728a137a2.jpg  
  inflating: prediction/prediction/d0bf0e1a756e2b76bc51355f8e7fd7bf.jpg  
  inflating: __MACOSX/prediction/prediction/._d0bf0e1a756e2b76bc51355f8e7fd7bf.jpg  
  inflating: prediction/prediction/22e043be47c72f87c4618a9b69ffbe91.jpg  
  inflating: __MACOSX/prediction/prediction/._22e043be47c72f87c4618a9b69ffbe91.jpg  
  inflating: prediction/prediction/90a2b6cd018abd1128b1b8075ab96a8a.jpg  
  inflating: __MACOSX/prediction/prediction/._90a2b6cd018abd1128b1b8075ab96a8a.jpg  
  inflating: prediction/prediction/b6e3fffc5c358ad65556d575db2fb317.jpg  
  inflating: __MACOSX/prediction/predict

In [0]:
def load_submission_dataset():
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    dataset = torchvision.datasets.ImageFolder(
        root='prediction',
        transform=transform
    )
    dataset_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
    return dataset_loader

In [0]:
model.eval()
predictions = []
with torch.no_grad():
    correct, total = 0, 0
    for images, labels in load_submission_dataset():
        outputs = model(images.cuda())
        _, result = torch.max(outputs.data, 1)
        predictions.append(result[0].item())

In [0]:
predictions = np.array(predictions)
df = pd.read_csv('sample_submission.csv')
image_id = df.id
submission = pd.DataFrame({'id':image_id, 'has_cactus':predictions})

In [0]:
submission.to_csv('submission.csv', index = False)